# Segmenting and Clustering Neighborhoods in Toronto 
## Applied Data Science Capstone
### By Andreia Queiroz Correia

**1) Import libraries**

In [31]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

**2) Create a dataframe will with three columns: PostalCode, Borough, and Neighborhood**

In [32]:
dfToronto = pd.DataFrame() 

col_names =  ["PostCode", "Borough", "Neihbourhood"]
dfToronto  = pd.DataFrame(columns = col_names)

**3) Fill the database with data of wikipedia**

In [33]:
import requests
from bs4 import BeautifulSoup

URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')
for items in soup.find('table', class_='wikitable sortable').find_all('tr')[1::1]:
    tds = items.find_all('td')
    if not tds:
        continue
    vPostCode, vBorough, vNeihbourhood = [td.text.strip() for td in tds[:3]]
    dfToronto.loc[len(dfToronto)] = [vPostCode, vBorough, vNeihbourhood]
 

**4) Ignore cells with a borough that is Not assigned.**

In [34]:
dfToronto = dfToronto[dfToronto.Borough != 'Not assigned']

**5) More than one neighborhood can exist in one postal code area.
   These two rows will be combined into one row with the neighborhoods separated with a comma**


In [35]:
grpPostCode = dfToronto.groupby(['PostCode'])

sNeihbourhood = ""
sBorough = ""
listToronto = []
    
for PostCode, Neihbourhood in grpPostCode:
              
    sNeihbourhood = ""
    sBorough = ""
                    
    for row, data in Neihbourhood.iterrows():  
        if sNeihbourhood != "":
            sNeihbourhood += " ,"
        sNeihbourhood += data['Neihbourhood']   
        sBorough = data['Borough']   

    listToronto.append([PostCode, sBorough, sNeihbourhood])

dfTorontoGRP = pd.DataFrame(listToronto)
dfTorontoGRP.columns = ["PostCode", "Borough", "Neihbourhood"]

**6) If a cell has  borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough**

In [36]:
dfTorontoGRP['Neihbourhood'].replace('Not assigned', dfTorontoGRP['Borough'], inplace=True)

**7) Display the first 5 rows**

In [37]:
dfTorontoGRP.head()

,PostCode,Borough,Neihbourhood
0,M1B,Scarborough,"Rouge ,Malvern"
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union"
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**8) Display the number of rows of your dataframe.**

In [38]:
dfTorontoGRP.shape

(103, 3)

**9) Load geographical coordinates**

In [39]:
dfGC = pd.read_csv('http://cocl.us/Geospatial_data')
dfGC.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [40]:
dfTorontoGRP["Latitude"] = dfGC[(dfGC['Postal Code']==dfTorontoGRP["PostCode"])]['Latitude']
dfTorontoGRP["Longitude"] = dfGC[(dfGC['Postal Code']==dfTorontoGRP["PostCode"])]['Longitude']

In [41]:
dfTorontoGRP.head()

,PostCode,Borough,Neihbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge ,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
